In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import itertools

In [59]:
def basis_generator(tot_sites, N_e): # Checked OK

  basis = []

  for combination_indices in itertools.combinations(range(tot_sites), N_e):
    state = [0] * tot_sites
    for index in combination_indices:
      state[index] = 1
    basis.append(state)
  
  return np.array(basis, dtype=np.float64)


def hamiltonian_generator_w_int_diff_V(basis_set, tot_sites, v, w, V1, V2):
  d = len(basis_set)
  H = np.zeros((d, d))

  for i in range(d):
    for j in range(d):
      H[j][i] = np.dot(hamiltonian_on_ket_w_int_diff_V(basis_set[i], tot_sites, v, w, V1, V2), basis_set[j])

  return H


def hamiltonian_on_ket_w_int_diff_V(ket, tot_sites, v, w, V1, V2):
  res = np.zeros((tot_sites), dtype=np.float64)
  for i in range(tot_sites):
    if i % 2 == 0 and i - 1 > 0:
      if ket[i - 1] == 1:
        res[i] += w
        if ket[i] == 1:
          res[i] += V2
      if ket[i + 1] == 1:
        res[i] += v
        if ket[i] == 1:
          res[i] += V1
    elif i % 2 == 1 and i + 1 < tot_sites:
      if ket[i - 1] == 1:
        res[i] += v
        if ket[i] == 1:
          res[i] += V1
      if ket[i + 1] == 1:
        res[i] += w
        if ket[i] == 1:
          res[i] += V2
    elif i == 0 and ket[i + 1] == 1:
      res[i] += v
      if ket[i] == 1:
          res[i] += V1
    elif i == tot_sites - 1 and ket[i - 1] == 1:
      res[i] += v
      if ket[i] == 1:
          res[i] += V1
  return res # normalize(res)


def normalize(vec):
  norm = np.linalg.norm(vec)
  if norm == 0:
    return vec
  return vec / norm

In [ ]:
def H_gen_test_v1(basis_set, tot_sites, v, w, V1, V2):
    d = len(basis_set)
    H = np.zeros((d, d))
    H_ket_arr = np.zeros((d, tot_sites))

    for i, ket in enumerate(basis_set):
        H_ket_arr[i] = H_ket(ket, tot_sites, v, w, V1, V2)
    
    for i in range(d):
        for j in range(d):
            H[i, j] = np.dot(H_ket_arr[i], basis_set[j])
    
    return H


def H_ket(ket, tot_sites, v, w, V1, V2):
    res = np.zeros((tot_sites))
    for i in range(tot_sites):
        if (i % 2 == 1) and (i != tot_sites - 1):
            if ket[i - 1] == 1:
                res[i] += v
                if ket[i] == 1:
                    res[i] += V1
            if ket[i + 1] == 1:
                res[i] += w
                if ket[i] == 1:
                    res[i] += V2
        elif (i % 2 == 0) and (i != 0):
            if ket[i - 1] == 1:
                res[i] += w
                if ket[i] == 1:
                    res[i] += V2
            if ket[i + 1] == 1:
                res[i] += v
                if ket[i] == 1:
                    res[i] += V1
        elif i == 0:
            if ket[i + 1] == 1:
                res[i] += v
                if ket[i] == 1:
                    res[i] += V1
        elif i == tot_sites - 1:
            if ket[i - 1] == 1:
                res[i] += v
                if ket[i] == 1:
                    res[i] += V1
    
    return res



In [57]:
N = 2
w = 2
v = 1
V1 = 3
V2 = 4
tot_sites = 2 * N
N_e = N

basis_set = basis_generator(tot_sites, N_e)

for i in range(len(basis_set)):
    ket = basis_set[i]
    print(f'{ket} ------ {H_ket(ket, tot_sites, v, w, V1, V2)}')

hamiltonian = H_gen_test_v1(basis_set, tot_sites, v, w, V1, V2)

print(hamiltonian)

print(sp.linalg.ishermitian(hamiltonian))

# e_val_arr, e_vec_arr = np.linalg.eigh(hamiltonian)

# for e_val in e_val_arr:
#     print(e_val)
  
# print(e_vec_arr)

[1. 1. 0. 0.] ------ [4. 4. 2. 0.]
[1. 0. 1. 0.] ------ [0. 3. 0. 1.]
[1. 0. 0. 1.] ------ [0. 1. 1. 0.]
[0. 1. 1. 0.] ------ [1. 6. 6. 1.]
[0. 1. 0. 1.] ------ [1. 0. 3. 0.]
[0. 0. 1. 1.] ------ [0. 2. 4. 4.]
[[ 8.  6.  4.  6.  4.  2.]
 [ 3.  0.  1.  3.  4.  1.]
 [ 1.  1.  0.  2.  1.  1.]
 [ 7.  7.  2. 12.  7.  7.]
 [ 1.  4.  1.  3.  0.  3.]
 [ 2.  4.  4.  6.  6.  8.]]
False


In [61]:
N = 2
w = 2
v = 1
V1 = 3
V2 = 4
tot_sites = 2 * N
N_e = N

basis_set = basis_generator(tot_sites, N_e)

for i in range(len(basis_set)):
    ket = basis_set[i]
    print(f'{ket} ------ {hamiltonian_on_ket_w_int_diff_V(ket, tot_sites, v, w, V1, V2)}')

hamiltonian = hamiltonian_generator_w_int_diff_V(basis_set, tot_sites, v, w, V1, V2)

print(hamiltonian)

print(sp.linalg.ishermitian(hamiltonian))

# e_val_arr, e_vec_arr = np.linalg.eigh(hamiltonian)

# for e_val in e_val_arr:
#     print(e_val)
  
# print(e_vec_arr)

[1. 1. 0. 0.] ------ [4. 4. 2. 0.]
[1. 0. 1. 0.] ------ [0. 3. 0. 1.]
[1. 0. 0. 1.] ------ [0. 1. 1. 0.]
[0. 1. 1. 0.] ------ [1. 6. 6. 1.]
[0. 1. 0. 1.] ------ [1. 0. 3. 0.]
[0. 0. 1. 1.] ------ [0. 2. 4. 4.]
[[ 8.  3.  1.  7.  1.  2.]
 [ 6.  0.  1.  7.  4.  4.]
 [ 4.  1.  0.  2.  1.  4.]
 [ 6.  3.  2. 12.  3.  6.]
 [ 4.  4.  1.  7.  0.  6.]
 [ 2.  1.  1.  7.  3.  8.]]
False
